In [1]:
import os
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [3]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok").is_between(1900,2024))
df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

794838
681475


In [8]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","260.parquet")), left_on="001", right_on="001", how="left")

In [16]:
df

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba,260_a,260_b,260_c,260_e,260_f,260_g,260_3,260_ind1
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str,list[str],list[str],list[str],list[str],list[str],str,list[str],list[str]
"""1""","""Molt, Petr""","""xx0043279""","[""aut"", ""ill""]",null,null,null,null,null,"""ck9101693""",""" nam a22 4500""","""910321s1991 xr a u0…","""(váz.)""",null,"[""80-204-0207-1""]",null,null,null,null,null,null,"""1""","""0""","""Toto shromáždění nebylo povole…","""svědectví fotoreportérů Mladé …","""[fotografie] Petr Molt, Herber…",null,null,null,null,null,"[""155 s. :""]","[""il. ;""]","[""30 cm""]",null,null,null,…,"[""1""]","[""Slavík, Herbert,""]","[""ill""]","[""1963-""]","[""jn20000710538""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1991,155,"""pevná""","[""Praha :""]","[""Mladá fronta,""]","[""1991""]",null,"[""(Svoboda 1)""]",null,null,null
"""1""","""Werich, Jan,""","""jk01151601""","[""aut""]","""1905-1980""",null,null,null,null,"""nkc20112174175""",""" nam a22 a 4500""","""110311s2011 xr ach e 0…","""(váz.)""",null,"[""978-80-7243-503-6""]",null,null,null,null,null,null,"""1""","""0""","""Listování""","""úryvky z korespondence a článk…","""Jan Werich ; [uspořádala, kome…",null,null,null,null,null,"[""285 s. :""]","[""il., portréty, faksim. ;""]","[""21 cm""]",null,null,null,…,"[""1""]","[""Bělíková, Marie""]","[""edt""]",null,"[""mzk2003169454""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2011,285,"""pevná""","[""Praha :""]","[""Brána,""]","[""2011""]",null,null,null,null,null
"""1""","""May, Karl,""","""jn19990005499""","[""aut""]","""1842-1912""",null,null,null,null,"""np9544870""",""" nam a22 4500""","""950809s1995 xr g 0…","""(váz.) :""","[""Kč 106,00""]","[""80-7174-805-6""]",null,null,null,null,null,null,"""1""","""0""","""Muž mnoha tváří""",null,"""Karel May ; [z němčiny přeloži…",null,null,null,null,null,"[""278 s. ;""]",null,"[""20 cm""]",null,null,null,…,"[""1"", ""1""]","[""Šunda, Vladimír"", ""Hanák, Jiří""]","[""trl"", ""art""]",null,"[""mzk2003187764"", ""jn19981001414""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1995,278,"""pevná""","[""Brno :""]","[""Návrat,""]","[""1995""]","[""(Brno :""]","[""Spektrum)""]",null,null,null
"""1""","""Tyrychtr, Zdeněk""","""jx20040604010""","[""aut""]",null,null,null,null,null,"""ck8402560""",""" nam a22 4500""","""840712s1984 xr a u0…","""(Brož.) :""","[""5,50 Kčs""]",null,null,null,null,null,null,null,"""1""","""0""","""Občanská nauka pro 1. ročník t…",null,"""Zdeněk Tyrychtr ; Josef Makal …",null,null,null,null,null,"[""110 s. :""]","[""fot., schémata, grafy ;""]","[""21 cm""]",null,null,null,…,"[""1"", ""1""]","[""Makal, Josef,"", ""Rotport, Miloslav,""]","[""aut"", ""aut""]","[""1931-"", ""1944-""]","[""jk01073140"", ""jn20001227035""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1984,110,"""brožovaná""","[""Praha :""]","[""SPN,""]","[""1984""]",null,"[""(Svoboda 5)""]",null,null,null
"""1""","""Hlaváček

In [18]:
df.filter(pl.col("245_a").str.contains("Jak s tím"))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba,260_a,260_b,260_c,260_e,260_f,260_g,260_3,260_ind1
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str,list[str],list[str],list[str],list[str],list[str],str,list[str],list[str]
"""1""","""Houška, Vítězslav,""","""jk01042486""","[""aut""]","""1925-2011""",null,null,null,null,"""bk196502371""",""" nam a22 1 4500""","""970909s1965 xr |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Jak s tím pohnu""",null,"""Vítězslav Houška ; [barev.] il…",null,null,null,null,null,"[""188 s. ;""]",null,"[""8°""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1965,188,null,"[""Praha :""]","[""SNDK,""]","[""1965""]",null,"[""(Polygrafia 1)""]",null,null,null


In [22]:
df = df.explode('260_b').with_columns(pl.col('260_b').map_elements(bez_bordelu, return_dtype=str))

In [32]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","830.parquet")), left_on="001", right_on="001", how="left")

In [34]:
df.filter(pl.col('245_a').str.contains('Jak s tím pohnu'))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba,260_a,260_b,260_c,260_e,260_f,260_g,260_3,260_ind1,830_ind2,830_a,830_p,830_n,830_t,830_s,830_o,830_v,830_d,830_x,830_m
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str,list[str],str,list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],str,str,str
"""1""","""Houška, Vítězslav,""","""jk01042486""","[""aut""]","""1925-2011""",null,null,null,null,"""bk196502371""",""" nam a22 1 4500""","""970909s1965 xr |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Jak s tím pohnu""",null,"""Vítězslav Houška ; [barev.] il…",null,null,null,null,null,"[""188 s. ;""]",null,"[""8°""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1965,188,null,"[""Praha :""]","""SNDK""","[""1965""]",null,"[""(Polygrafia 1)""]",null,null,null,"[""0""]","[""Krystal (Státní nakladatelství dětské knihy)""]",null,null,null,null,null,null,null,null,null


In [26]:
df.filter(pl.col("260_b") == "SNDK").drop_nulls(["700_a",'700_7','700_4']).explode(["700_a",'700_7','700_4'])

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba,260_a,260_b,260_c,260_e,260_f,260_g,260_3,260_ind1
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str,list[str],str,list[str],list[str],list[str],str,list[str],list[str]
"""1""","""Štruncová, Olga,""","""jk01131543""","[""ant""]","""1926-""",null,null,null,null,"""bk196503839""",""" nam a22 1 4500""","""971102s1965 xr …",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Hledej a najdeš""","""Pro nejmenší /""","""Návrh Olgy Štruncové výtvarně …",null,null,null,null,null,"[""1 sv. :""]","[""[11] s. barev. obr. ;""]","[""příč. 8°""]",null,null,null,…,"[""1""]","""Pospíšil, Antonín,""","""ill""","[""1909-2008""]","""jk01100658""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1965,1,null,"[""Praha :""]","""SNDK""","[""1965""]","[""(Velký Šenov :""]","[""SG 2)""]",null,null,null
"""1""","""Andersen, Hans Christian,""","""jn19990000171""","[""aut""]","""1805-1875""",null,null,null,null,"""bk195801900""",""" nam a22 1 4500""","""980314s1957 xr …","""(váz.) :""","[""Kčs 40,00""]",null,null,null,null,null,null,null,"""1""","""0""","""Pohádky""",null,"""[Autor:] Hans Christian Anders…",null,null,null,null,null,"[""248, [5] s. ;""]",null,"[""4°""]",null,null,null,…,"[""1""]","""Trnka, Jiří,""","""ill""","[""1912-1969""]","""jk01140140""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1957,248,"""pevná""","[""Praha :""]","""SNDK""","[""1957""]",null,"[""(Impressa, z.z.)""]",null,null,null
"""1""","""Kriebel, Zdeněk,""","""jk01063328""","[""aut""]","""1911-1989""",null,null,null,null,"""bk196603451""",""" nam a22 1 4500""","""971013s1966 xr |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Koulej se, sluníčko, kutálej""",null,"""Zdeněk Kriebel ; [Barev.] il. …",null,null,null,null,null,"[""168. [5] s. ;""]",null,"[""8°""]",null,null,null,…,"[""1""]","""Trnka, Jiří,""","""ill""","[""1912-1969""]","""jk01140140""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1966,168,null,"[""Praha :""]","""SNDK""","[""1966""]",null,"[""(Polygrafia 1)""]",null,null,null
"""1""","""Jirásek, Alois,""","""jk01051816""","[""aut""]","""1851-1930""",null,null,null,null,"""bk195401692""",""" nam a22 1 4500""","""980228s1954 xr …","""(Brož.)""",null,null,null,null,null,null,null,null,"""1""","""0""","""Na Chlumku""","""[Mimočítanková četba pro 5. po…","""Alois Jirásek ; [ilustroval Mi…",null,null,null,null,null,"[""31, [5] s. :""]","[""il. ;""]","[""8°""]",null,null,null,…,"[""1""]","""Aleš, Mikoláš,""","""ill""","[""1852-1913""]","""jk01010178""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1954,31,"""brožovaná""","[""Praha :""]","""SNDK""","[""1954""]",null,"[""(Knihtisk, z.z. 01)""]",null,null,null
"""1""","""Štruncová, Olga,""","""jk01131543""","[""aut""]","""1926-""",null,null,null,null,"""bk195301821""",""" nam a22 1 4500""","""980302s1953 xr |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Hledej a najdeš!""",null,"""Olga Štruncová, Antonín Pospíš…",null,null,null

In [28]:
df.filter(pl.col("260_b") == "SNDK").drop_nulls(["700_a",'700_7','700_4']).explode(["700_a",'700_7','700_4']).filter(pl.col('700_4') == 'ill').group_by('700_a').len().sort(by='len',descending=True)

700_a,len
str,u32
"""Trnka, Jiří,""",25
"""Pospíšil, Antonín,""",21
"""Hanák, Mirko,""",18
"""Zmatlíková, Helena,""",18
"""Junek, Václav,""",14
"""Marešová, Milada,""",14
"""Kudělka, Zdeněk,""",12
"""Aleš, Mikoláš,""",11
"""Fuka, Vladimír,""",10


In [38]:
df.filter(pl.col("260_b") == "SNDK").explode("830_a").filter(pl.col("830_a").str.contains("Krystal"))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba,260_a,260_b,260_c,260_e,260_f,260_g,260_3,260_ind1,830_ind2,830_a,830_p,830_n,830_t,830_s,830_o,830_v,830_d,830_x,830_m
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str,list[str],str,list[str],list[str],list[str],str,list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],str,list[str],str,str,str
"""1""","""Tichý, Jaroslav,""","""jk01132421""","[""aut""]","""1923-2004""",null,null,null,null,"""bk196505128""",""" nam a22 1 4500""","""971203s1965 xr |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Letem ČSSR""",null,"""Jaroslav Tichý ; [Barev.] il. …",null,null,null,null,null,"[""214, [6] s. ;""]",null,"[""příč. 8°""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1965,214,null,"[""Praha :""]","""SNDK""","[""1965""]",null,"[""(Svoboda 2)""]",null,null,null,"[""0"", ""0""]","""Krystal (Státní nakladatelství…","[null, ""Výběrová řada""]",null,null,null,null,null,null,null,null
"""1""","""Houška, Vítězslav,""","""jk01042486""","[""aut""]","""1925-2011""",null,null,null,null,"""bk196502371""",""" nam a22 1 4500""","""970909s1965 xr |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Jak s tím pohnu""",null,"""Vítězslav Houška ; [barev.] il…",null,null,null,null,null,"[""188 s. ;""]",null,"[""8°""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1965,188,null,"[""Praha :""]","""SNDK""","[""1965""]",null,"[""(Polygrafia 1)""]",null,null,null,"[""0""]","""Krystal (Státní nakladatelství…",null,null,null,null,null,null,null,null,null
"""1""","""Koval, Václav,""","""jk01062409""","[""aut""]","""1907-""",null,null,null,null,"""bk196205560""",""" nam a22 1 4500""","""981102s1962 xr c 0…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Na dně vzdušného moře""",null,"""[Autoři:] Václav Koval, Petr H…",null,null,null,null,null,"[""221, [4] s. ;""]",null,"[""příč. 8°""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1962,221,null,"[""Praha :""]","""SNDK""","[""1962""]","[""(Liberec :""]","[""SG)""]",null,null,null,"[""0""]","""Krystal (Státní nakladatelství…",null,null,null,null,null,null,null,null,null
"""1""","""Hons, Josef,""","""jk01041936""","[""com""]","""1907-2001""",null,null,null,null,"""bk196703688""",""" nam a22 1 4500""","""971008s1967 xr |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""A štafeta běží dál""",null,"""Josef Hons ; il. Dobroslav Fol…",null,null,null,null,null,"[""214, [34] s. :""]","[""[6] barev. obr. příl. ;""]","[""8°""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1967,214,null,"[""Praha :""]","""SNDK""","[""1967""]",null,"[""(Svoboda 2)""]",null,null,null,"[""0""]","""Krystal (Státní nakladatelství…",null,null,null,null,null,null,null,null,null
"""1""","""Štuka, Ivo,""","""jk01131550""","[""aut""]","""1930-2007""",null,null,null,null,"""bk196300694""",""" nam a22 1 4500""","""971119s1963 xr |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Šest dnů na luně 1""",null,"""Ivo Štuka ; [Barev.] il. Teodo…",null,null,null,null,null,"[""200, [3] s. ;""]",null,"[""příč. 8°""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1963,200,null,

In [44]:
df.filter(pl.col("245_a").str.contains("Rychlé šípy")).drop_nulls(["700_a",'700_7','700_4']).explode(["700_a",'700_7','700_4']).filter(pl.col('700_4') == 'ill').group_by(['700_a','700_7']).len().sort(by='len',descending=True)

700_a,700_7,len
str,str,u32
"""Fischer, Jan,""","""jn19981228074""",15
"""Čermák, Marko,""","""jn20001005428""",8
"""Teslevič, Milan,""","""mzk2006322665""",2
"""Čermák, Bohumír,""","""mzk2010580489""",2
"""Grus, Jiří,""","""mzk2005289872""",1
"""Junek, Václav,""","""jk01052158""",1
"""Zeman, Karel,""","""xx0062777""",1


In [46]:
df.explode('700_7').filter(pl.col('700_7') == 'jn19981228074')

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba,260_a,260_b,260_c,260_e,260_f,260_g,260_3,260_ind1,830_ind2,830_a,830_p,830_n,830_t,830_s,830_o,830_v,830_d,830_x,830_m
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str,list[str],str,list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],str,str,str
"""1""","""Foglar, Jaroslav,""","""jk01031485""","[""aut""]","""1907-1999""",null,null,null,null,"""cpk20051498549""",""" nam a22 a 4500""","""050523s2005 xr a g 0…","""(váz.)""",null,"[""80-7033-870-9"", ""978-80-7033-870-4""]",null,null,null,null,null,null,"""1""","""0""","""Tajemství Velkého Vonta""",null,"""Jaroslav Foglar ; [ilustrace J…",null,null,null,null,null,"[""204 s. :""]","[""il. ;""]","[""21 cm""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2005,204,"""pevná""","[""Praha :""]","""Olympia""","[""2005""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""1""","""Foglar, Jaroslav,""","""jk01031485""","[""aut""]","""1907-1999""",null,null,null,null,"""nkc20233551157""",""" nam a22 i 4500""","""231003s2020 xr ah c 0…","""(vázáno) :""","[""Kč 499,00""]","[""978-80-00-07140-4""]",null,null,null,null,null,null,"""1""","""0""","""Záhada hlavolamu""","""přepodivný příběh ze života Ry…","""Jaroslav Foglar ; ilustroval J…",null,null,null,null,null,"[""254, xix stran :""]","[""ilustrace (převážně barevné), faksimile ;""]","[""25 cm""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2020,254,"""pevná""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""1""","""Foglar, Jaroslav,""","""jk01031485""","[""aut""]","""1907-1999""",null,null,null,null,"""ck9104936""",""" nam a22 4500""","""910917s1991 xr a u0…","""(Brož.)""",null,null,null,null,null,null,null,null,"""1""","""0""","""Rychlé šípy 60x""","""soutěže, závody, hry a ostatní…","""Jaroslav Foglar ; sestavil Iva…",null,null,null,null,null,"[""94 s. :""]","[""barev. obr. ;""]","[""29 cm""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1991,94,"""brožovaná""","[""Praha :""]","""Comet""","[""1991""]",null,"[""(Svoboda)""]",null,null,null,"[""0""]","[""Magazín Komety""]",null,null,null,null,null,null,null,null,null
"""1""","""Foglar, Jaroslav,""","""jk01031485""","[""aut""]","""1907-1999""",null,null,null,null,"""nkc20213314569""",""" cam a22 i 4500""","""210329s2021 xr ah c 6 0…","""(vázáno)""","[""Kč 899,00""]","[""978-80-00-06123-8""]","[""9788000061238 :""]",null,null,null,null,null,"""1""","""0""","""Rychlé šípy Jaroslava Foglara …",null,null,null,null,null,null,null,"[""289 stran :""]","[""ilustrace (převážně barevné), faksimile ;""]","[""34 cm""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2021,289,"""pevná""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""1""","""Foglar, Jaroslav,""","""jk01031485""","[""aut""]","""1907-1999""",null,null,null,null,"""np9540250""",""" nam a22 4500""","""950823s1995 xr a j u0…","""(váz.) :""","[""Kč 79,00""]","[""80-7033-372-3""]",null,null,null,null,null,null,"""1""","""0""","""Stínadla se bouří""",null,"""Jaroslav Foglar ; [ilustroval …",null,null,nul

In [54]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","650.parquet")), left_on="001", right_on="001", how="left")

In [62]:
def hezky(frejm):
    return frejm.select(pl.col(['100_a','100_7','245_a','245_b','rok','700_a','700_4'])).sort(by="rok")

In [64]:
hezky(df.explode("650_7").filter(pl.col("650_7") == "ph858666"))

100_a,100_7,245_a,245_b,rok,700_a,700_4
str,str,str,str,i64,list[str],list[str]
"""Brus, Jan,""","""jo2013770943""","""Vizualizace nejistoty v enviro…","""autoreferát disertační práce =…",2013,null,null
"""Brus, Jan,""","""jo2013770943""","""Vizualizace nejistoty v enviro…",null,2014,null,null
"""Gemignani, Zach,""","""xx0196245""","""Efektivní analýza a využití da…",null,2015,"[""Gemignani, Chris"", ""Galentino, Richard"", … ""Huf, Jiří""]","[""aut"", ""aut"", … ""trl""]"
"""Šafařík, Josef,""","""jx20050429020""","""Multimediální design - interpr…","""Multimedia design - interpreta…",2020,null,null
"""Barvíř, Radek,""","""mzk20211100555""","""Metriky grafické náplně map za…","""autoreferát dizertační práce =…",2021,null,null
"""Barvíř, Radek,""","""mzk20211100555""","""Metriky grafické náplně map za…",null,2022,null,null
"""Trahorsch, Petr""","""xx0227431""","""Vizuálie v učebnicích geografi…","""analýza a vliv na dětská pojet…",2022,null,null


In [66]:
df.explode("650_7").filter(pl.col("650_7") == "ph858666").explode('650_x').group_by('650_x').len().sort(by='len',descending=True)

650_x,len
str,u32
null,7


In [142]:
def souvisejici(tema):
    pracovni = df.explode("650_a").filter(pl.col("650_a") == tema)
    knihy = pracovni.select(pl.col('008')).to_series().to_list()
    temata = df.filter(pl.col('008').is_in(knihy)).explode('650_a').group_by('650_a').len().sort(by='len',descending=True).head(20)
    print("SOUVISEJÍCÍ TÉMATA\n\n" + str(temata.head(10)))
    
    aut_knih = pracovni.filter(pl.col('650_a').is_in(temata.select(pl.col('650_a')).to_series().to_list())).group_by("100_a").len().sort(by='len',descending=True).head(10)
    print("PROLIFICKÉ AUTORSTVO\n\n" + str(aut_knih))

In [146]:
souvisejici('grafy a schémata')

SOUVISEJÍCÍ TÉMATA

shape: (10, 2)
┌─────────────────────────────────┬─────┐
│ 650_a                           ┆ len │
│ ---                             ┆ --- │
│ str                             ┆ u32 │
╞═════════════════════════════════╪═════╡
│ grafy a schémata                ┆ 16  │
│ charts, diagrams, etc           ┆ 7   │
│ charts, diagrams, etc.          ┆ 5   │
│ Microsoft Visio 2003 (software… ┆ 3   │
│ pedagogický výzkum              ┆ 3   │
│ educational research            ┆ 3   │
│ digital photography             ┆ 2   │
│ správa počítačových sítí        ┆ 2   │
│ spoje (komunikace)              ┆ 2   │
│ Canon EOS 1000D                 ┆ 2   │
└─────────────────────────────────┴─────┘
PROLIFICKÉ AUTORSTVO

shape: (10, 2)
┌────────────────────┬─────┐
│ 100_a              ┆ len │
│ ---                ┆ --- │
│ str                ┆ u32 │
╞════════════════════╪═════╡
│ Kuba, Zdeněk,      ┆ 2   │
│ Svoboda, Emanuel,  ┆ 2   │
│ Munroe, Randall,   ┆ 1   │
│ Smrčina, Otomar,   ┆ 1

In [144]:
souvisejici('kartografie')

SOUVISEJÍCÍ TÉMATA

shape: (10, 2)
┌────────────────────────────┬─────┐
│ 650_a                      ┆ len │
│ ---                        ┆ --- │
│ str                        ┆ u32 │
╞════════════════════════════╪═════╡
│ kartografie                ┆ 78  │
│ cartography                ┆ 48  │
│ mapy                       ┆ 14  │
│ maps                       ┆ 10  │
│ geodézie                   ┆ 10  │
│ zeměpis                    ┆ 8   │
│ null                       ┆ 7   │
│ geography (school subject) ┆ 7   │
│ geoinformatika             ┆ 6   │
│ geoinformatics             ┆ 6   │
└────────────────────────────┴─────┘
PROLIFICKÉ AUTORSTVO

shape: (10, 2)
┌────────────────────┬─────┐
│ 100_a              ┆ len │
│ ---                ┆ --- │
│ str                ┆ u32 │
╞════════════════════╪═════╡
│ Pyšek, Jiří,       ┆ 4   │
│ Vondráková, Alena, ┆ 4   │
│ Kuchař, Karel,     ┆ 3   │
│ Piska, Rudolf,     ┆ 2   │
│ Rogl, Vladimír,    ┆ 2   │
│ Maršík, Zbyněk,    ┆ 2   │
│ Šíma, Jiří,    

In [82]:
def podobne(tema):
    knihy = df.explode("650_a").filter(pl.col("650_a") == tema).select(pl.col('008')).to_series().to_list()
    return df.filter(pl.col('008').is_in(knihy)).explode('650_a').group_by('650_a').len().sort(by='len',descending=True).head(50)

In [84]:
podobne('vizualizace informací')

650_a,len
str,u32
"""information visualization""",7
"""vizualizace informací""",7
"""kartografie""",4
"""spatial data""",4
"""prostorová data""",4
"""cartography""",4
"""data quality""",2
"""geoinformatics""",2
"""user experience design""",2


In [88]:
podobne("data analysis")

650_a,len
str,u32
"""data analysis""",76
"""analýza dat""",76
"""R (software)""",20
"""data processing""",14
"""zpracování dat""",14
"""statistická analýza""",14
"""statistical analysis""",14
"""statistické metody""",9
"""statistical methods""",9


In [98]:
podobne("dotazníky")

650_a,len
str,u32
"""dotazníky""",56
"""questionnaires""",51
"""psychologický výzkum""",6
"""statistical analysis""",5
"""klima školy""",5
"""statistická analýza""",5
"""datové soubory""",5
"""data analysis""",5
"""data files""",5


In [90]:
podobne("geostatistics"	)

650_a,len
str,u32
"""geostatistika""",7
"""geostatistics""",7
"""prostorová data""",5
"""spatial data""",5
"""geographical information syste…",4
"""statistické mapy""",4
"""labor market""",4
"""trh práce""",4
"""statistical maps""",4


In [92]:
podobne("webdesign")

650_a,len
str,u32
"""webdesign""",154
"""web design""",151
"""webové stránky""",53
"""web pages""",51
"""kaskádové styly""",34
"""cascading style sheets""",34
"""programování""",32
"""programming""",32
"""dynamické webové aplikace""",21


In [96]:
podobne("information architecture")

650_a,len
str,u32
"""user experience design""",2
"""webdesign""",1
"""information architecture""",1
"""interakce člověk-počítač""",1
"""human-computer interaction""",1
"""web design""",1
"""informační architektura""",1


In [86]:
podobne('user experience design')

650_a,len
str,u32
"""user experience design""",4
"""interakce člověk-počítač""",2
"""human-computer interaction""",2
"""webdesign""",1
"""dokumentární film""",1
"""information architecture""",1
"""user interfaces""",1
"""perception of art""",1
"""film themes""",1
